In [1]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.6/740.6 kB 18.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 26.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30/30 [langchain]30 [langgraph]core]
Note: you may need to restart the kernel to use updated packages.


In [20]:
from dotenv import load_dotenv
import os

In [34]:
load_dotenv("/home/ubuntu/project/.env")

True

In [37]:
API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
from openai import OpenAI

In [38]:
client = OpenAI(api_key = API_KEY)

### 랭체인 사용 Open ai 호출하기

In [39]:
import os
os.environ["PYTHONIOENCODING"] = "utf-8"

In [ ]:
pip install -U "openai>=1.51.0" "httpx>=0.27.0" "langchain>=0.3.0" "langchain-openai>=0.2.0"


In [4]:
from dotenv import load_dotenv, find_dotenv
import os
from langchain_openai import ChatOpenAI

# .env 로드
load_dotenv(find_dotenv(), override=True)

api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "OPENAI_API_KEY 가 없습니다 (.env 확인)"
assert all(ord(c) < 128 for c in api_key), "API 키에 비-ASCII 문자가 섞였습니다"

# LangChain 클라이언트: 조직/프로젝트/프록시/UA를 깨끗하게
llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=api_key,
    # organization=None, project=None  # 필요한 경우 명시 가능
    default_headers={"User-Agent": "langchain-openai"},  # ASCII만
)

resp = llm.invoke("하늘은 왜 파란가요?")
print(resp.content)


하늘이 파란 이유는 대기 중의 산란 과정 때문입니다. 태양 빛은 여러 색의 빛으로 구성되어 있으며, 이 색들은 각각 다른 파장을 가지고 있습니다. 파장이 짧은 파란색 빛은 대기 중의 분자와 입자에 의해 다른 색보다 더 많이 산란됩니다. 

해가 높이 떠 있을 때, 태양 빛이 대기를 통과하면서 파란색 빛이 다양한 방향으로 퍼져 나가고, 그 결과 우리는 하늘이 파랗게 보이게 됩니다. 이 현상을 '레이리 산란'이라고 부릅니다. 반대로, 해가 지거나 떠오를 때, 태양 빛은 더 많은 대기를 통과하게 되므로 빨강, 주황색과 같은 긴 파장의 색이 더 많이 산란되어 하늘이 붉거나 주황색으로 보이기도 합니다.


In [45]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="너는 친절한 조교야"),
    HumanMessage(content="안녕?"),
    AIMessage(content="안녕하세요! 무엇을 도와드릴까요?"),
    HumanMessage(content="Langchain이 뭐야?")
]

response = llm.invoke(messages)
print(response.content)

LangChain은 언어 모델(예: OpenAI의 GPT와 같은)을 사용하는 애플리케이션을 구축하는 데 도움을 주는 프레임워크입니다. 주로 자연어 처리(NLP) 작업을 보다 효과적으로 수행할 수 있도록 도와주는 여러 구성 요소와 도구를 제공합니다.

LangChain의 주요 기능은 다음과 같습니다:

1. **체인**: 여러 단계를 연결하여 복잡한 작업을 수행할 수 있는 방법을 제공합니다. 예를 들어, 모델의 출력을 다른 API 호출에 전달하거나 저장된 데이터를 검색하는 방식으로 체인을 구성할 수 있습니다.

2. **메모리**: 사용자의 입력이나 모델의 출력을 저장하고 재사용할 수 있는 메커니즘을 제공합니다. 이를 통해 보다 일관된 대화나 인터랙션을 구현할 수 있습니다.

3. **로드 체인**: 외부 데이터 소스(예: 데이터베이스, 웹 API 등)에 접근하여 모델이 더 나은 응답을 생성할 수 있도록 지원합니다.

4. **프롬프트 관리**: 프롬프트 템플릿을 작성하고 관리하여 일관성 있게 모델에 입력할 수 있도록 지원합니다.

LangChain은 주로 대화형 애플리케이션 개발, 정보 검색, 문서 생성 등 다양한 작업에서 강력한 도구로 사용됩니다. 궁금한 점이나 더 알고 싶은 내용이 있으면 말씀해 주세요!


### 멀티턴 대화 구현하기

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [5]:
#세션별 대화 기록을 저장할 딕셔너리
store = {}

#세션 id에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id:str):
    #만약 세션 id가 없으면 새로 생성해 추가
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

#모델 실행시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_messages_history = RunnableWithMessageHistory(llm, get_session_history)

In [6]:
print(type(with_messages_history))

<class 'langchain_core.runnables.history.RunnableWithMessageHistory'>


In [7]:
print(with_messages_history)

bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]) kwargs={} config={} config_factories=[] get_session_history=<function get_session_history at 0xffff7183ec20> history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)]


In [8]:
config = {"configurable": {"session_id" : "chat-001"}} #세션 id를 설정하는 config 객체 생성

response = with_messages_history.invoke(
    [HumanMessage(content="안녕? 나는 홍길동이야")],
    config = config
)

print(response.content)

안녕하세요, 홍길동님! 만나서 반가워요. 어떻게 도와드릴까요?


In [9]:
config

{'configurable': {'session_id': 'chat-001'}}

In [10]:
response = with_messages_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config = config
)

print(response.content)

당신의 이름은 홍길동입니다. 다른 질문이나 도움이 필요하신 부분이 있나요?


In [11]:
store

{'chat-001': InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕? 나는 홍길동이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 16, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CT6MqwzbTaUqvkl7C3bJlK0rw6Onm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5ab128f3-915a-4b1d-a7d2-5f152f7f4337-0', usage_metadata={'input_tokens': 16, 'output_tokens': 22, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), Hu

In [12]:
config = {"configurable": {"session_id" : "chat-002"}} #세션 id를 설정하는 config 객체 생성

response = with_messages_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config = config
)

print(response.content)

죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 기쁠 것 같습니다!


In [13]:
store

{'chat-001': InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕? 나는 홍길동이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 16, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CT6MqwzbTaUqvkl7C3bJlK0rw6Onm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5ab128f3-915a-4b1d-a7d2-5f152f7f4337-0', usage_metadata={'input_tokens': 16, 'output_tokens': 22, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), Hu

In [14]:
config = {"configurable": {"session_id" : "chat-001"}} #세션 id를 설정하는 config 객체 생성

response = with_messages_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 이야기 했지?")],
    config = config
)

print(response.content)

우리는 처음에 당신의 이름이 홍길동이라는 걸 이야기했어요. 그 외에 다른 특정한 대화는 없었죠. 더 이야기하고 싶은 주제가 있으면 말씀해 주세요!


In [15]:
store

{'chat-001': InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕? 나는 홍길동이야', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 16, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CT6MqwzbTaUqvkl7C3bJlK0rw6Onm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5ab128f3-915a-4b1d-a7d2-5f152f7f4337-0', usage_metadata={'input_tokens': 16, 'output_tokens': 22, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), Hu